# Peg Solitaire Solver
### El Aamrani Ahmed - Lhourti Wassim

In [1]:
import numpy as np
import cvxopt
import cvxopt.glpk
from cvxopt import matrix
import time

In [2]:
#Starting reprensentation
ps = np.ones(33)
ps[16]=0 #Central hole has no peg in the starting representation

#Final representation
pf = np.zeros(33)
pf[16]=1 #Central hole is the only filled hole in the final representation

#Defining 'l' parameter
l = int(np.sum(ps)-np.sum(pf))

#Defining 'm' parameter
m = 76 # It corresponds to the number of possible jumps in an English peg solitaire problem

#Defining cost_matrix
cost_matrix = matrix(np.zeros(l*m), tc='d')

#Defining binary variables
B = set(range(l*m))

In [3]:
#Implementing matrix A (please refer to the report for its definition)

#First, implementing A1
A10= np.array([[1,0,0,0,0], [1,1,0,0,0], [-1,1,1,0,0], [0,-1,1,1,0], [0,0,-1,1,1], [0,0,0,-1,1], [0,0,0,0,-1]])

A11 = np.zeros((33,1))
A11[0]=1
A11[1]=1
A11[2]=-1

A12 = np.zeros((33,1))
A12[3]=1
A12[4]=1
A12[5]=-1

A13 = np.vstack((np.zeros((6,5)), A10, np.zeros((20,5))))

A14 = np.vstack((np.zeros((13,5)), A10, np.zeros((13,5))))

A15 = np.vstack((np.zeros((20,5)), A10, np.zeros((6,5))))

A16 = np.zeros((33,1))
A16[27]=1
A16[28]=1
A16[29]=-1

A17 = np.zeros((33,1))
A17[30]=1
A17[31]=1
A17[32]=-1

A1 = np.hstack((A11,A12,A13,A14,A15,A16,A17))


#Then, implementing A2
A20= np.array([[-1,0,0,0,0], [1,-1,0,0,0], [1,1,-1,0,0], [0,1,1,-1,0], [0,0,1,1,-1], [0,0,0,1,1], [0,0,0,0,1]])

A21 = np.zeros((33,1))
A21[0]=-1
A21[1]=1
A21[2]=1

A22 = np.zeros((33,1))
A22[3]=-1
A22[4]=1
A22[5]=1

A23 = np.vstack((np.zeros((6,5)), A20, np.zeros((20,5))))

A24 = np.vstack((np.zeros((13,5)), A20, np.zeros((13,5))))

A25 = np.vstack((np.zeros((20,5)), A20, np.zeros((6,5))))

A26 = np.zeros((33,1))
A26[27]=-1
A26[28]=1
A26[29]=1

A27 = np.zeros((33,1))
A27[30]=-1
A27[31]=1
A27[32]=1

A2 = np.hstack((A21,A22,A23,A24,A25,A26,A27))


#Then, implementing A3
A31 = np.vstack((-np.eye(3), np.eye(3), np.zeros((2,3)), np.eye(3), np.zeros((22,3))))

A32 = np.vstack((np.zeros((3,3)), -np.eye(3), np.zeros((2,3)), np.eye(3), np.zeros((4,3)), np.eye(3), np.zeros((15,3))))

A33 = np.vstack((np.zeros((6,7)), -np.eye(7), np.eye(7), np.eye(7), np.zeros((6,7))))

A34 = np.vstack((np.zeros((15,3)), -np.eye(3), np.zeros((4,3)), np.eye(3), np.zeros((2,3)), np.eye(3), np.zeros((3,3))))

A35 = np.vstack((np.zeros((22,3)), -np.eye(3), np.zeros((2,3)), np.eye(3), np.eye(3)))

A3 = np.hstack((A31,A32,A33,A34,A35))


#Then, implementing A4
A41 = np.vstack((np.eye(3), np.eye(3), np.zeros((2,3)), -np.eye(3), np.zeros((22,3))))

A42 = np.vstack((np.zeros((3,3)), np.eye(3), np.zeros((2,3)), np.eye(3), np.zeros((4,3)), -np.eye(3), np.zeros((15,3))))

A43 = np.vstack((np.zeros((6,7)), np.eye(7), np.eye(7), -np.eye(7), np.zeros((6,7))))

A44 = np.vstack((np.zeros((15,3)), np.eye(3), np.zeros((4,3)), np.eye(3), np.zeros((2,3)), -np.eye(3), np.zeros((3,3))))

A45 = np.vstack((np.zeros((22,3)), np.eye(3), np.zeros((2,3)), np.eye(3), -np.eye(3)))

A4 = np.hstack((A41,A42,A43,A44,A45))

# Concatenating A_i horizontally to constitute A
A = np.hstack((A1,A2,A3,A4)).astype(int)

In [4]:
# let us define equality matrices. There are two kind of equality constraints. 

# Let us consider first equality constraints (4) - Please refer to the report
E11 = np.hstack((np.eye(m) for i in range(l)))
E1 = np.matmul(A,E11)

e1 = (ps - pf).reshape((33,1))

# Let us consider the second king of equality constraints (6) - Please refer to the report
E2 = np.zeros((l,l*m))
for i in range(l):
    E2[i][(i*m):((i+1)*m)] = np.ones((1,m))

e2 = np.ones((l,1))

# Concatenating vertically equality matrices
E = np.vstack((E1,E2))
E = matrix(E, tc='d')

e = np.vstack((e1,e2))
e = matrix(e, tc='d')

In [5]:
# let us define inequality matrices. There are one kind (5) of equality constraints with 2 sides.

# Let us consider first the " >= 0 " side
for k in range(l):
    I1k1 = np.hstack((np.eye(m) for i in range(k+1)))
    if k == l-1:
        I1k = I1k1
    else:
        I1k2 = np.hstack((np.zeros((m,m)) for j in range(k+1,l)))
        I1k = np.hstack((I1k1,I1k2))
    A1k = np.matmul(A,I1k)
    if k == 0:
        I1 = A1k
    else:
        I1 = np.vstack((I1, A1k))

i10 = ps.reshape((33,1))
i1 = np.vstack((i10 for  i in range(l)))

# Let us consider first the " <= 0 " side
for k in range(l):
    I2k1 = np.hstack((-1*np.eye(m) for i in range(k+1)))
    if k == l-1:
        I2k = I2k1
    else:
        I2k2 = np.hstack((np.zeros((m,m)) for j in range(k+1,l)))
        I2k = np.hstack((I2k1,I2k2))
    A2k = np.matmul(A,I2k)
    if k == 0:
        I2 = A2k
    else:
        I2 = np.vstack((I2, A2k))

i20 = np.ones((33,1)) - ps.reshape((33,1))
i2 = np.vstack((i20 for  i in range(l)))

#Concatenating inequality matrices
I = np.vstack((I1,I2))
I = matrix(I, tc='d')

i = np.vstack((i1,i2))
i = matrix(i, tc='d')

In [6]:
# Verifying conditions for a feasible solution taken from "https://www.youtube.com/watch?v=Bt6GpGvUNeQ"
# Wa translated this feasible solution into our notations

# Each row corresponds to a sequence among l=31 sequences
# Each column is equivalent to the corresponding jump in the set J of cardinal 76

# The sequence of youtube video moves 
youtube_sol = [61,3,57,41,19,57,51,12,32,63,12,34,56,72,18,56,62,25,5,50,25,15,49,24,22,64,13,27,52,28,8]

# Defining the solution matrix
youtube_solution = np.zeros((l,m))
youtube_solution[np.arange(l), youtube_sol]  = np.ones(l)

#Reshape the solution to a vector instead of a matrix
youtube_solution = youtube_solution.reshape((l*m,1))

print("--------------------------------------------------- Validation step ---------------------------------------------------")
print("This feasible solution verifies the 'IP1' equality constraints :", np.array_equal(np.matmul(E,youtube_solution),e))
print("This feasible solution verifies the 'IP1' inequality constraints :", np.sum(np.matmul(I,youtube_solution)<=i)==I.size[0])
print("-----------------------------------------------------------------------------------------------------------------------")
print("You can change any move within the sequence and see that these constraints are no longer true !")

--------------------------------------------------- Validation step ---------------------------------------------------
This feasible solution verifies the 'IP1' equality constraints : True
This feasible solution verifies the 'IP1' inequality constraints : True
-----------------------------------------------------------------------------------------------------------------------
You can change any move within the sequence and see that these constraints are no longer true !


In [7]:
# Solving the problem

#(status, optimal_x) = cvxopt.glpk.ilp(c=cost_matrix,G=I,h=i,A=E,b=e,B=B) (***)

print("-----------------------------------------------------------------------------------------------------------------------")
print("The above line of code (***) is an implmentation for the \"IP1\" integer program formulation")
print("You could run this code but we've realized that it takes an infinite time of computing")
print("Thus, the necessity of exploring other ways to solve this problem.")
print("-----------------------------------------------------------------------------------------------------------------------")

-----------------------------------------------------------------------------------------------------------------------
The above line of code (***) is an implmentation for the "IP1" integer program formulation
You could run this code but we've realized that it takes an infinite time of computing
Thus, the necessity of exploring other ways to solve this problem.
-----------------------------------------------------------------------------------------------------------------------


In [8]:
# An alternative way is to implement a backtrack search using upper_bounds for the number of specific moves (using RUB)
# and using pagoda function


# Here we shuffle columns of A, in order to explore randomly moves among the 76 possible ones
# When fixing seed=82, we have better performances
np.random.seed(seed=82)
s = np.arange(A.shape[1])
np.random.shuffle(s)
A = A[:,s]


# Defining matrices for RUB integer program
Erub = matrix(A, tc='d')
    
Irub = -1*np.eye(m)
Irub = matrix(Irub, tc='d')

irub = np.zeros((m,1))
irub = matrix(irub, tc='d')

Srub = set(range(m))


# Taking an example of 3 pagoda functions that were given in the original paper
pagval1_array = np.array([1,0,1,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,1,0,1])
pagval2_array = np.array([0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0])
pagval3_array = np.array([0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0])

# Computing pagoda value for the finishing configuration
pagval1_end = np.matmul(pagval1_array.T,pf)
pagval2_end = np.matmul(pagval2_array.T,pf)
pagval3_end = np.matmul(pagval3_array.T,pf)

In [9]:
# Defining a function that solves RUB program given the jump j, the starting and finishing configurations

def solveRUB(jump, starting_config, finishing_config):
    cost = np.zeros(m)
    cost[jump] = -1
    cost = matrix(cost, tc='d')
    
    erub = matrix((starting_config - finishing_config).reshape((33,1)), tc='d')
    
    (status, optimal_x) = cvxopt.glpk.ilp(c=cost,G=Irub,h=irub,A=Erub,b=erub,I=Srub)
    
    if optimal_x:
        x_jump = np.matmul(cost.T, optimal_x)
        x_jump = x_jump[0][0]
        return(-1*x_jump)
    else:
        return None

In [10]:
# Defining a function which states if a start configuration (actually an intermediate configuration) is legal or not
# meaning wether it is a binary vector or not, knowing that its elements are integers

def legal(start):
    for i in range(len(start)):
        if start[i]<0 or start[i]>1:
            return False
    return True

In [11]:
# Defining a recursive function that takes in input the intermediate configuration "start", the finishing configuration "end
# which remains always the same, the list of upper bounds, the dictionary of searches already computed, and the path which is
# updated at each stage.

def search(start, end, upper_bound, path, dictionary):
    
    reste = int(np.sum(start)-np.sum(end))
    
    if reste <= 0:
        return(np.array_equal(start,end))
    
    else:
        pagval1_t = np.matmul(pagval1_array.T,start)
        pagval2_t = np.matmul(pagval2_array.T,start)
        pagval3_t = np.matmul(pagval3_array.T,start)

        if pagval1_t < pagval1_end or pagval2_t < pagval2_end or pagval3_t < pagval3_end:
            return False
        
        else:
            
            for j in range(m):
                
                optimal_x = solveRUB(j, start, end)
                
                if optimal_x is None:
                    return False
                
                else:
                    if upper_bound[j] <= 0:
                        continue

                    u_j = np.zeros(m)
                    u_j[j] = 1

                    move = start - np.matmul(A,u_j)

                    if legal(move):
                        start = move

                        upper_bound[j] = upper_bound[j] - 1
                        path.append(s[j])

                        mytuple = hash(tuple(start))

                        if mytuple in dictionary.keys():
                            y = dictionary[mytuple]
                        else:
                            y = search(start, end, upper_bound, path, dictionary)
                            dictionary[mytuple] = y

                        if y == 1:
                            return True
                        else:
                            upper_bound[j] = upper_bound[j] + 1
                            start = start + np.matmul(A,u_j)
                            path.pop()

            return False

In [12]:
# Defining the backtrack peg solver

def backtrack_peg_solver(starting_config, finishing_config):
    dictionary = {}
    upper_bound = np.zeros(m)
    path = []
    
    for jump in range(m):
        optimal_x = solveRUB(jump, starting_config, finishing_config)
        if optimal_x is None:
            print("Infeasible problem")
            return None
        else:
            upper_bound[jump] = optimal_x
    
    if search(starting_config, finishing_config, upper_bound, path, dictionary) == 0:
        print("Infeasible problem")
        return None
    
    print("Feasible problem")
    return path

In [13]:
start = time.time()
solution = backtrack_peg_solver(ps, pf)
end = time.time()
print("The list of moves composing the feasible solution is : \n-->", solution)
print("The elapsed computing time is :", round(end - start,2), "s")

Feasible problem
The list of moves composing the feasible solution is : 
--> [61, 47, 13, 54, 32, 55, 12, 32, 63, 12, 21, 34, 32, 56, 72, 62, 45, 2, 22, 57, 41, 19, 57, 25, 50, 3, 5, 25, 67, 53, 29]
The elapsed computing time is : 3.25 s


In [14]:
#Verifying whether the solution verifies the \"IP1\" integer program formulation that was first defined in the Notebook

feasible_sol = np.zeros((l,m))
# 'solution' is the list of moves 
feasible_sol[np.arange(l),solution]  = np.ones(l)

#Reshape the feasible solution to a vector instead of a matrix
feasible_sol = feasible_sol.reshape((l*m,1))

print("--------------------------------------------------- Validation step ---------------------------------------------------")
print("This solution verifies the 'IP1' equality constraints :", np.array_equal(np.matmul(E,feasible_sol),e))
print("This solution verifies the 'IP1' inequality constraints :", np.sum(np.matmul(I,feasible_sol)<=i)==I.size[0])
print("-----------------------------------------------------------------------------------------------------------------------")

--------------------------------------------------- Validation step ---------------------------------------------------
This solution verifies the 'IP1' equality constraints : True
This solution verifies the 'IP1' inequality constraints : True
-----------------------------------------------------------------------------------------------------------------------


In [15]:
print("-----------------------------------------------------------------------------------------------------------------------")
print("This peg solitaire solver is adapted for any starting and finishing configuration which are defined in 'In[2]'")
print("-----------------------------------------------------------------------------------------------------------------------")

-----------------------------------------------------------------------------------------------------------------------
This peg solitaire solver is adapted for any starting and finishing configuration which are defined in 'In[2]'
-----------------------------------------------------------------------------------------------------------------------


In [16]:
#Graphical representation of the solution 

#As A has been shuffled before, we recreate the initial A matrix
A = np.hstack((A1,A2,A3,A4)).astype(int)

def print_config(p):
    #This function gives a graphical representation of a given configuration p
    
    #Convert p to a string type array
    str_p = np.char.mod('%d', p)
    #Replace integers by black and white dots
    for i in range(len(str_p)):
        if str_p[i] == '0':
            str_p[i] = 'o'
        elif str_p[i] == '1':
            str_p[i] = '\u25CF'
        else:
            return(False)
    #Convert the array to a string
    string_sol = ''.join(str_p)
    #Represent each line
    first_line = '              ' + string_sol[:3] + '  '  
    second_line = '              ' + string_sol[3:6] + '  '
    third_line = '            ' + string_sol[6:13]  
    fourth_line = '            ' + string_sol[13:20]
    fifth_line = '            ' + string_sol[20:27]
    sixth_line = '              ' + string_sol[27:30] + '  '
    seventh_line = '              ' + string_sol[30:] + '  '
    #Concatenate lines
    config = first_line + '\n' + second_line + '\n' + third_line + '\n' + fourth_line + '\n' + fifth_line + '\n' + sixth_line + '\n' + seventh_line + '\n'
    print(config)
    #return(config)

def print_solution(solution):
    #This function prints the solution's sequence
    print('------Initial Configuration' + '------\n')
    #result = []
    print_config(ps)
    p = ps
    zeros = np.zeros(m)
    for i in range(len(solution)-1):
        zeros[solution[i]] = 1
        p = p - np.matmul(A,zeros)
        print('-------------Move ' + str(i+1) +'-------------\n')
        print_config(p)
        #result.append(print_config(p))
        zeros = np.zeros(m)
    print('------Final Configuration' + '------\n')
    print_config(pf)
    #result.append(print_config(pf))
    #str3 = ' '.join(result)
    #print(str3)
    

print_solution(solution)


------Initial Configuration------

              ●●●  
              ●●●  
            ●●●●●●●
            ●●●o●●●
            ●●●●●●●
              ●●●  
              ●●●  

-------------Move 1-------------

              ●●●  
              ●o●  
            ●●●o●●●
            ●●●●●●●
            ●●●●●●●
              ●●●  
              ●●●  

-------------Move 2-------------

              ●●●  
              ●o●  
            ●●●●●●●
            ●●●o●●●
            ●●●o●●●
              ●●●  
              ●●●  

-------------Move 3-------------

              ●●●  
              ●o●  
            ●●●●●●●
            ●●●o●●●
            ●oo●●●●
              ●●●  
              ●●●  

-------------Move 4-------------

              ●●●  
              ●o●  
            ●●●●●●●
            ●●●o●●●
            ●o●●●●●
              o●●  
              o●●  

-------------Move 5-------------

              ●●●  
              ●o●  
            ●●●●●●●
            ●●●o●●●
          